In [1]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='BTC-Bitstamp-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Timestamp": "dateid", "Close": "price","Volume_(BTC)":"volume"})


df

,dateid,Open,High,Low,price,volume,Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [2]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume','High','Low'])
df

,price,ask,bid,volume,High,Low
date,,,,,,
2011-12-31 07:52:00,4.39,4.4339,4.3461,0.455581,4.39,4.39
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,58714.31,58686.00
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,58693.43,58683.97
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,58723.84,58693.43


In [3]:
#df = df.dropna()

def delta(timeseriesShort, timeseriesLong):
    return ((timeseriesShort/timeseriesLong)-1).clip(-3,3)

def addMa(timeframe):
    p=df['price']
    df['ma'+timeframe] = delta(p,p.rolling(timeframe).mean())

def add2Ma(timeframeshort, timeframelong):
    p=df['price']
    df['ma'+timeframeshort+'_'+timeframelong] =delta(p.rolling(timeframeshort).mean(),p.rolling(timeframelong).mean())


def addMinMax(timeframe,periods,freq):
    p=df['price']
    dfs = df.shift(periods=periods, freq=freq)
    mins = dfs['Low'].rolling(timeframe).min()
    df['maxmin'+timeframe]=((p - mins)/(dfs['High'].rolling(timeframe).max() - mins)).clip(-3,3)
    

def addBol(timeframe):
    p=df['price']
    df['bol'+timeframe] = (delta(p,p.rolling(timeframe).mean())/p.rolling(timeframe).std()).clip(-3,3)

def addStdDir(timeframe,periods,freq):
    p=df['price']
    val=p.rolling(timeframe).std()
    df['stddir'+timeframe] = delta(val,val.shift(periods=periods, freq=freq))

def addMaDir(timeframe,periods,freq):
    p=df['price']
    val = p.rolling(timeframe).mean()
    df['madir'+timeframe] = delta(val,val.shift(periods=periods, freq=freq))







addMa('5min')
addMa('1h')
addMa('5D')
addBol('20D')
addMinMax('10D',1,"D")
addBol('5D')
add2Ma('6min','13min')
add2Ma('6h','13h')

df = df.dropna()

    
del df['High']
del df['Low']

df

,price,ask,bid,volume,ma5min,ma1h,ma5D,bol20D,maxmin10D,bol5D,ma6min_13min,ma6h_13h
date,,,,,,,,,,,,
2012-01-01 15:28:00,4.84,4.8884,4.7916,10.000000,0.000000,0.000000,0.061792,0.373183,3.000000,0.373183,0.000000,2.760085e-02
2012-01-01 22:45:00,5.00,5.0500,4.9500,10.100000,0.000000,0.000000,0.081917,0.363753,3.000000,0.363753,0.000000,1.626016e-02
2012-01-02 20:04:00,5.00,5.0500,4.9500,19.048000,0.000000,0.000000,0.070950,0.286370,1.355556,0.286370,0.000000,1.110223e-15
2012-01-03 11:45:00,5.32,5.3732,5.2668,2.419173,0.000000,0.000000,0.122100,0.384509,1.524590,0.384509,0.000000,1.110223e-15
2012-01-03 14:22:00,5.14,5.1914,5.0886,0.680000,0.000000,0.000000,0.075089,0.231132,1.229508,0.231132,0.000000,4.440892e-16
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,-0.000404,0.000310,0.049769,0.000017,0.867563,0.000027,0.000530,-1.043274e-03
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,-0.000325,0.000313,0.049747,0.000017,0.867543,0.000027,0.000361,-1.040359e-03
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,0.000236,0.000956,0.050408,0.000017,0.871506,0.000027,0.000238,-1.038407e-03


In [4]:
df['bol15min'].max()

KeyError: 'bol15min'